In [1]:
from vis_utils import load_volume, VolumeVisualizer, ColorMapVisualizer
from identification import get_vessel_graph, remove_graph_components, parametrize_graph, clean_data_graph

In [2]:
import numpy as np
import networkx as nx
import time
import psutil

from queue import SimpleQueue
from skimage import measure, morphology 
from scipy.ndimage import distance_transform_edt, zoom
from scipy.signal import fftconvolve
from scipy.spatial.distance import euclidean

In [3]:
visualize_steps = False 

### Visualisation utilities

In [4]:
def visualize_mask_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8), binary=True).visualize()
    
def visualize_mask_non_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8) * 255, binary=False).visualize()
    
def draw_graph_on_model(binary_model, graph):
    mask = np.zeros(binary_model.shape, dtype=np.uint8)
    mask[binary_model] = 30
    

    for edge in graph.edges:
        node1, node2 = edge
        x1, y1, z1 = node1
        x2, y2, z2 = node2
        line_x = np.linspace(x1, x2, num=300, endpoint=True, dtype=np.int32)
        line_y = np.linspace(y1, y2, num=300, endpoint=True, dtype=np.int32)
        line_z = np.linspace(z1, z2, num=300, endpoint=True, dtype=np.int32)
        for i in range(len(line_x)):
            mask[line_x[i], line_y[i], line_z[i]] = 255

    for node in graph.nodes:
        x, y, z = node
        mask[x, y, z] = 40
    return mask

In [5]:
import re

def get_volume(filename):
    try:
        tokens = re.split(r'x|_|\.', filename)
        shape_z, shape_y, shape_x = int(tokens[-4]), int(tokens[-3]), int(tokens[-2])
        volume = np.fromfile(filename, dtype=np.uint8)
        return volume.reshape(shape_x, shape_y, shape_z)
    except:
        print("Invalid filename, correct format: <filename>_<shape x>x<shape y>x<shape z>.raw")

### Read data

In [6]:
volume = get_volume("../data/P13/P13_60um_1132x488x877.raw")

In [7]:
volume.shape

(877, 488, 1132)

In [8]:
if visualize_steps:
    VolumeVisualizer(volume, binary=False).visualize() 

In [9]:
mask = volume > 32

In [10]:
def get_main_regions(binary_mask, min_size=10_000, connectivity=3):
    labeled = measure.label(binary_mask, connectivity=connectivity)
    region_props = measure.regionprops(labeled)
    
    main_regions = np.zeros(binary_mask.shape)
    bounding_boxes = []
    for props in region_props:
        if props.area >= min_size:
            bounding_boxes.append(props.bbox)
            main_regions = np.logical_or(main_regions, labeled==props.label)
            
    lower_bounds = np.min(bounding_boxes, axis=0)[:3]
    upper_bounds = np.max(bounding_boxes, axis=0)[3:]

    return main_regions[
        lower_bounds[0]:upper_bounds[0],
        lower_bounds[1]:upper_bounds[1],
        lower_bounds[2]:upper_bounds[2],
    ], bounding_boxes

In [11]:
main_regions, bounding_boxes = get_main_regions(mask, min_size=25_000)

In [12]:
print('Number of main regions:', len(bounding_boxes))

Number of main regions: 1


In [13]:
if visualize_steps:
    visualize_mask_bin(main_regions)

In [14]:
def spherical_kernel(outer_radius, thickness=1, filled=True):    
    outer_sphere = morphology.ball(radius=outer_radius)
    if filled:
        return outer_sphere
    
    thickness = min(thickness, outer_radius)
    
    inner_radius = outer_radius - thickness
    inner_sphere = morphology.ball(radius=inner_radius)
    
    begin = outer_radius - inner_radius
    end = begin + inner_sphere.shape[0]
    outer_sphere[begin:end, begin:end, begin:end] -= inner_sphere
    return outer_sphere

def convolve_with_ball(img, ball_radius, dtype=np.uint16, normalize=True, fft=True):
    kernel = spherical_kernel(ball_radius, filled=True)
    if fft:
        convolved = fftconvolve(img.astype(dtype), kernel.astype(dtype), mode='same')
    else:
        convolved = signal.convolve(img.astype(dtype), kernel.astype(dtype), mode='same')
    
    if not normalize:
        return convolved
    
    return (convolved / kernel.sum()).astype(np.float16)

def calculate_reconstruction(mask, kernel_sizes=[10, 9, 8, 7], fill_threshold=0.5, iters=1, conv_dtype=np.uint16, fft=True):
    kernel_sizes_maps = []
    mask = mask.astype(np.uint8)
    
    for i in range(iters):
        kernel_size_map = np.zeros(mask.shape, dtype=np.uint8)

        for kernel_size in kernel_sizes:
            fill_percentage = convolve_with_ball(mask, kernel_size, dtype=conv_dtype, normalize=True, fft=fft)
            
            above_threshold_fill_indices = fill_percentage > fill_threshold
            kernel_size_map[above_threshold_fill_indices] = kernel_size + 1

            mask[above_threshold_fill_indices] = 1
            
        kernel_sizes_maps.append(kernel_size_map)
        print(f'Iteration {i + 1} ended successfully')

    return kernel_sizes_maps

In [15]:
scaled_mask = zoom(main_regions, zoom=0.7, order=0)

In [16]:
scaled_mask.shape

(598, 324, 775)

In [17]:
if visualize_steps:
    visualize_mask_bin(scaled_mask)

### Initial reconstruction

In [ ]:
%%time
s_recos = calculate_reconstruction(scaled_mask, 
                                   kernel_sizes=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
                                   iters=3)

Iteration 1 ended successfully
Iteration 2 ended successfully


In [ ]:
bin_reco = s_recos[-1] > 0

In [ ]:
if visualize_steps:
    visualize_mask_bin(bin_reco)

### EDT transform

In [ ]:
%%time
edt_img = distance_transform_edt(bin_reco)

In [ ]:
if visualize_steps:
    ColorMapVisualizer(edt_img.astype(np.uint8)).visualize()

### Fing vertices and edges

In [ ]:
%%time
use_cube = False
vessel_graph = get_vessel_graph(bin_reco, 3, use_cube)

In [ ]:
print('Number of nodes', len(vessel_graph.nodes))
print('Number of edges', len(vessel_graph.edges))
print('Average degree', sum(dict(vessel_graph.degree).values()) / len(vessel_graph.nodes))

In [ ]:
import pickle
pickle.dump(vessel_graph, open('get_vessel_full_2.pickle', 'wb'))

In [ ]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(bin_reco, vessel_graph)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

### Remove zero-degree vertices

In [ ]:
%%time
vessel_graph_rm = remove_graph_components(vessel_graph)

In [ ]:
print('Number of nodes', len(vessel_graph_rm.nodes))
print('Number of edges', len(vessel_graph_rm.edges))
print('Average degree', sum(dict(vessel_graph_rm.degree).values()) / len(vessel_graph_rm.nodes))

In [ ]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(bin_reco, vessel_graph_rm)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

### Original connect endings

In [ ]:
def connect_endings_3d(graph, edt_img, multiplier=2.5):
    """Fix discontinuities in vessel graph.

    Args:
        graph (nx.Graph): Graph object from NetworkX library.
        edt_img (np.array): Numpy array containing euclidean distance transformed image of vessel.
        multiplier (int, optional): Multiplier for maximum distance between nodes to connect.
        Defaults to 2.

    Returns:
        nx.Graph: Graph without discontinuities.
    """
    # find potential discontinuities
    endings = [node for node, degree in graph.degree() if degree == 1]
    
    # for every ending run BFS connection search
    print("Endings: ", len(endings))
    while endings:
        # get point to find connection for
        start = endings.pop()
        # calculate search area
        r = edt_img[start] * multiplier
        search_area = int(4/3 * r * r * r * np.pi)
        # setup BFS
        points = SimpleQueue()
        points.put(start)
        visited = np.zeros_like(edt_img).astype(bool)
        
        # run BFS on a restricted area
        while not points.empty() and search_area:
            search_area -= 1
            # get point
            x, y, z = points.get()
            visited[x, y, z] = True
            
            # check if point is a node and is a valid connection
            if graph.has_node((x, y, z)) and not nx.has_path(graph, start, (x, y, z)):
                graph.add_edge(start, (x, y, z))
                # this is to prevent accidentally creating bifurcations
                if (x, y, z) in endings:
                    endings.pop(endings.index((x, y, z)))
                break

            # add point to search if it is in segmentation mask and it is not visited
            for dx in range(-1, 2):
                for dy in range(-1, 2):
                    for dz in range(-1, 2):
                        new_point = (x + dx, y + dy, z + dz)
                        if (
                            x + dx >= 0
                            and x + dx < edt_img.shape[0]
                            and y + dy >= 0
                            and y + dy < edt_img.shape[1]
                            and z + dz >= 0
                            and z + dz < edt_img.shape[2]
                            and not visited[new_point[0], new_point[1], new_point[2]]
                            and edt_img[x + dx, y + dy, z + dz] > 0
                        ):
                            visited[new_point[0], new_point[1], new_point[2]] = True
                            points.put(new_point)

    return graph

### Simplified connected

In [ ]:
def connect_endings_3d_dist(graph, edt_img, multiplier=3.):
    """Fix discontinuities in vessel graph.

    Args:
        graph (nx.Graph): Graph object from NetworkX library.
        edt_img (np.array): Numpy array containing euclidean distance transformed image of vessel.
        multiplier (int, optional): Multiplier for maximum distance between nodes to connect.
        Defaults to 2.

    Returns:
        nx.Graph: Graph without discontinuities.
    """
    # find potential discontinuities
    endings = [node for node, degree in graph.degree() if degree == 1]
    root = np.unravel_index(np.argmax(edt_img), edt_img.shape)
    
    if root not in graph.nodes:
        graph.add_node(root)
        
    print("Endings: ", len(endings))
    j = 0
    while endings:
        if j % 10 == 0:
            print(j)
        start = endings.pop()
        r = edt_img[start] * multiplier
        search_area = int(4/3 * r * r * r * np.pi)

        visited = np.zeros_like(edt_img).astype(bool)

        to_visit = sorted_nodes_by_distance(graph, start)
        i = 0
        while i < len(to_visit):
            if nx.has_path(graph, root, start) or euclidean(start, to_visit[i]) > edt_img[start] * multiplier:
                break
            curr = to_visit[i]
            i += 1
            if not nx.has_path(graph, start, curr) and curr != start:
                graph.add_edge(start, curr)
                if curr in endings:
                    endings.pop(endings.index(curr))
                break
        j += 1
    return graph

def sorted_nodes_by_distance(graph, node):
    distances = {}
    for n in list(graph.nodes):
        if n == node:
            continue
        dist = euclidean(node, n)
        distances[n] = dist
    sorted_nodes = sorted(distances, key=distances.get)
    return sorted_nodes

In [ ]:
%%time
vessel_graph_cn = connect_endings_3d(vessel_graph, edt_img, 3)

In [ ]:
# %%time
# vessel_graph_cn = connect_endings_3d_dist(vessel_graph, edt_img, 3)

In [ ]:
# vessel_graph = pickle.load(open("get_vessel_full_2.pickle", "rb"))

In [ ]:
print('Number of nodes', len(vessel_graph_cn.nodes))
print('Number of edges', len(vessel_graph_cn.edges))
print('Average degree', sum(dict(vessel_graph_cn.degree).values()) / len(vessel_graph_cn.nodes))

In [ ]:
#pickle.dump(vessel_graph_cn, open('connect_endings_3d_full.pickle', 'wb'))
pickle.dump(vessel_graph_cn, open('connect_endings_3d_full_dist_2.pickle', 'wb'))

In [ ]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(bin_reco, vessel_graph_cn)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [ ]:
visualize_steps = True

### Parametrize graph

In [ ]:
%%time
data_graph = parametrize_graph(vessel_graph_cn, edt_img)

In [ ]:
print('Number of nodes', len(data_graph.nodes))
print('Number of edges', len(data_graph.edges))
print('Average degree', sum(dict(data_graph.degree).values()) / len(data_graph.nodes))

In [ ]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(bin_reco, data_graph)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

### Clean graph

In [ ]:
def clean_data_graph(data_graph, min_segment_length=20):
    """Clean data graph by removing small segments, nodes of degree 2 and selecting a new root.

    Args:
        data_graph (nx.Graph): Undirected vessel data graph.
        min_segment_length (int, optional): Minimum segment length permissible in graph.
        Defaults to 20[px].

    Returns:
        nx.Graph: Cleaned vessel data graph.
    """
    # remove small segments
    for edge, segment_length in nx.get_edge_attributes(
        data_graph, "segment_length"
    ).items():
        if segment_length < min_segment_length:
            data_graph.remove_edge(*edge)
    # remove isolated nodes (degree == 0)
    data_graph.remove_nodes_from(list(nx.isolates(data_graph)))
    # merge segments for nodes of degree 2
    while [n for n, degree in data_graph.degree() if degree == 2]:
        for node, degree in list(data_graph.degree()):
            if degree != 2:
                continue
            n1, n2 = list(data_graph.neighbors(node))
            new_edge_data = {}
            new_edge_data["nodes"] = data_graph[node][n1]["nodes"]
            new_edge_data["nodes"].update(data_graph[node][n2]["nodes"])
            
            new_edge_data["segment_length"] = sum(
                node_data["vessel_length"]
                for node_data in new_edge_data["nodes"].values()
            )
            new_edge_data["average_vessel_diameter"] = sum(
                node_data["vessel_diameter"]
                for node_data in new_edge_data["nodes"].values()
            ) / len(new_edge_data["nodes"])
            data_graph.remove_node(node)
            data_graph.add_edge(n1, n2, **new_edge_data)
    # select new root
    root_node = choose_root_node(data_graph)
    nx.set_node_attributes(data_graph, False, "root")
    data_graph.nodes[root_node]["root"] = True
    return data_graph

In [ ]:
def choose_root_node(data_graph):
    """Select root node for data graph.

    Args:
        data_graph (nx.Graph): Undirected vessel data graph.

    Returns:
        tuple: coordinates of root node.
    """
    # select edge with biggest average vessel diameter
    edges = [
        k
        for k, _ in sorted(
            nx.get_edge_attributes(data_graph, "average_vessel_diameter").items(),
            key=lambda item: item[1],
        )
    ]
    n1, n2 = edges[-1]
    # select root by node degree
    if data_graph.degree(n1) == 1 and data_graph.degree(n2) != 1:
        return n1
    if data_graph.degree(n2) == 1 and data_graph.degree(n1) != 1:
        return n2
    # select root by biggest vessel diameter
    n1_diameter = data_graph[n1][n2]["nodes"][n1]["vessel_diameter"]
    n2_diameter = data_graph[n1][n2]["nodes"][n2]["vessel_diameter"]
    return n1 if n1_diameter > n2_diameter else n2

In [ ]:
%%time
data_graph_cl = clean_data_graph(data_graph)

In [ ]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(bin_reco, data_graph_cl)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [ ]:
print('Number of nodes', len(data_graph.nodes))
print('Number of edges', len(data_graph.edges))
print('Average degree', sum(dict(data_graph.degree).values()) / len(data_graph.nodes))

In [ ]:
model_with_graph = draw_graph_on_model(bin_reco, data_graph_cl)
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [ ]:
pickle.dump(data_graph_cl, open('clean_data_graph_full_2.pickle', 'wb'))